# Setup

In [106]:
#! pip install pandas-datareader
#! pip install pmdarima
#! pip install plotly
!pip install dash-bootstrap-components
import numpy as np
from pandas_datareader import DataReader # pip install pandas-datareader
from pandas_datareader import data
from datetime import datetime
from pmdarima.arima import *
from pmdarima import preprocessing
from scipy import stats
from scipy.stats import skew

from statsmodels import api as sm
from statsmodels.tsa.seasonal import seasonal_decompose

import pandas as pd
import pmdarima as pm
import plotly.graph_objects as go
import plotly.express as px
 
start = pd.to_datetime('2010-01-01') # in YYYY-MM-DD format
end = pd.to_datetime('2018-01-01')
ts = data.DataReader('NDAQ', 'yahoo', start , end) # here 'yahoo' is the API to yahoo

train = ts.Close.loc['2010-01-01':'2018-01-01']
test_7 = ts.Close.loc['2017-12-20': '2018-01-01']
test_31 = ts.Close.loc['2017-11-15':'2018-01-01']
two_month = ts.Close.loc['2017-10-15':'2018-01-01']

# Normalisation

In [107]:
ts.Close.skew()

0.4854340176287619

In [108]:
# put all normalisation functions before modelling

In [109]:
#transformed data
boxCoxData, boxCox_lambda = stats.boxcox(ts.Close)
boxCoxSkew = skew(boxCoxData)

#we want to compare their absolute skewness 
if(boxCoxSkew < 0):
    boxCoxSkew = boxCoxSkew * -1

johnsonData, johnson_lambda = stats.yeojohnson(ts.Close)
johnsonSkew = skew(johnsonData)

if(johnsonSkew < 0):
    johnsonSkew = johnsonSkew * -1

#to show which is less skewed 
print(johnsonSkew)
print(boxCoxSkew)

0.043066305774378665
0.0391631256848633


In [110]:
#so we know what method was used, so we can reverse later. 
normaliseMethod = "neither"

if(johnsonSkew < boxCoxSkew):
    normalised_Data = johnsonData
    fitted_lambda = johnson_lambda
    normaliseMethod = "johnson"
    
#being == doesn't really matter same either way
elif(boxCoxSkew <= johnsonSkew):
    normalised_Data = boxCoxData
    fitted_lambda = boxCox_lambda
    normaliseMethod = "boxcox"

skew(normalised_Data)
#print(fitted_lambda)

0.0391631256848633

In [111]:
actual = px.line(ts.Close, x=ts.index, y="Close", title="NASDAQ Closing Values: 2010 - 2018")
actual.show()

actual_hist = px.histogram(ts, title="Normal Distribution", x="Close", width=800, height=500)
actual_hist.show()

bc = pd.DataFrame(normalised_Data, columns = ['Close'])
bc = px.histogram(bc, x="Close")
bc.show()

# Modelling Functions

## Stationarity Tests:

In [112]:
def testLevelStationarity(ts):
    adf_result = sm.tsa.stattools.adfuller(ts)
    kpps_result = sm.tsa.stattools.kpss(ts)
    print(f'ADF p-value: {adf_result[1]}')
    print(f'KPPS p-value: {kpps_result[1]}')

    d = ndiffs(ts, test='kpss')
    d += ndiffs(ts, test='adf')
    d += ndiffs(ts, test='pp')
    
    return int(d/3)

In [113]:
testLevelStationarity(ts.Close)

ADF p-value: 0.976170301796687
KPPS p-value: 0.01


C:\Users\Oisin Tong\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1875: FutureWarning:

The behavior of using nlags=None will change in release 0.13.Currently nlags=None is the same as nlags="legacy", and so a sample-size lag length is used. After the next release, the default will change to be the same as nlags="auto" which uses an automatic lag length selection method. To silence this warning, either use "auto" or "legacy"

C:\Users\Oisin Tong\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1906: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.




1

## Seasonal Tests:

In [114]:
def seasonal_tests(data):
     
    result1 = pm.arima.nsdiffs(data, m=3, max_D=30, test='ch')
    
    print("CH results: " + str(result1))
    
    result2 = pm.arima.nsdiffs(data, m=3, max_D=30, test='ocsb')
    
    print("OCSB results: " + str(result2))
    
    return int((result1+result2)/2)


In [115]:
seasonal_tests(normalised_Data)

CH results: 0
OCSB results: 0


0

## Lag Period:

In [116]:
# create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return pd.DataFrame (diff,columns=['Difference'])

In [117]:
def find_lag_period(data):
    
    fig = px.line(ts.Close, x=[ts.Close.index], y="Close")
    fig.add_vline(x='2010-01-01', line_width=3, line_dash="dash", line_color="green")
    fig.add_vline(x='2011-01-01', line_width=3, line_dash="dash", line_color="green")
    fig.add_vline(x='2012-01-01', line_width=3, line_dash="dash", line_color="green")
    fig.add_vline(x='2013-01-01', line_width=3, line_dash="dash", line_color="green")
    fig.add_vline(x='2014-01-01', line_width=3, line_dash="dash", line_color="green")
    fig.add_vline(x='2015-01-01', line_width=3, line_dash="dash", line_color="green")
    fig.add_vline(x='2016-01-01', line_width=3, line_dash="dash", line_color="green")
    fig.add_vline(x='2017-01-01', line_width=3, line_dash="dash", line_color="green")
    fig.add_vline(x='2018-01-01', line_width=3, line_dash="dash", line_color="green")
    fig.show()
    
    adf_test = ADFTest(alpha = 0.05)
    
    test = list()
    lags = [365, 182, 90, 30, 14]
    
    # original data
    test.append(adf_test.should_diff(data))
    print(test[0])
    
    # year lag
    test.append(adf_test.should_diff(difference(data, 365)))
    print(test[1])
    
    # six months lag
    test.append(adf_test.should_diff(difference(data, 182))) 
    print(test[2])
    
    # three months lag 
    test.append(adf_test.should_diff(difference(data, 90)))
    print(test[3])
    
    # one month lag 
    test.append(adf_test.should_diff(difference(data, 30)))
    print(test[4])
    
    # fortnight lag
    test.append(adf_test.should_diff(difference(data, 14))) 
    print(test[5])
    
    # finds the first lag that doesn't need to be differenced
    for i in range(6):
        if (test[i][1] == False):
            return int(lags[i])
        
    return -1

In [118]:
print(find_lag_period(normalised_Data))
print(find_lag_period(ts.Close))

(0.08966768756968181, True)
(0.45042711876766006, True)
(0.08009108397189417, True)
(0.01002064977478713, False)
(0.01, False)
(0.01, False)
30


(0.3917697604899978, True)
(0.3792055847549318, True)
(0.026625072978079433, False)
(0.01, False)
(0.01, False)
(0.01, False)
90


## AR & MA Tests

In [119]:
def getAicBicHqic(dataset, arimaOrder, seasonalOrder):
    
    try:
        model = sm.tsa.statespace.SARIMAX(dataset, order = arimaOrder, seasonal_order=seasonalOrder).fit(disp=False)

        aic = model.aic
        bic = model.bic
        hqic = model.hqic
        
    
    except:
        pass
    
    return aic, bic, hqic

In [120]:
def evaluateSarimaModels(dataset, pVals, dVal, qVals, seasonalPVals, seasonalDval, seasonalQVals, m):
    
    L = []
    for p in pVals:
        for q in qVals:
            for seasonalP in seasonalPVals:
                for seasonalQ in seasonalQVals:
                    try:
                        order=(p,dVal,q)
                        seasonalOrder = (seasonalP, seasonalDval, seasonalQ, m)
                        
                        model = sm.tsa.statespace.SARIMAX(dataset, order = order, seasonal_order=seasonalOrder).fit(disp=False)

                        data = [[model, (p,dVal,q), (seasonalP, seasonalDval, seasonalQ, m), model.aic, model.bic, model.hqic, model.mse, model.test_heteroskedasticity(None)[0][0]]]
                        dd = pd.DataFrame(data = data, columns = ['model','Order', 'SeasonalOrder', 'Aic', 'Bic', 'Hqic', 'Mse', 'heteroskedasticity'])
                        L.append(dd)
                    except:
                        continue
    df = pd.concat(L, ignore_index=True)
    return df



In [121]:
df = evaluateSarimaModels(normalised_Data, [0,1], 1, [0,1], [0,1], 1, [0,1], 7)


C:\Users\Oisin Tong\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\Oisin Tong\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\Oisin Tong\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\Oisin Tong\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\Oisin Tong\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\Oisin Tong\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to con

In [122]:
df_johnson = evaluateSarimaModels(johnsonData, [0,1], 1, [0,1], [0,1], 1, [0,1], 7)

C:\Users\Oisin Tong\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\Oisin Tong\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\Oisin Tong\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\Oisin Tong\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



In [123]:
def sarima_model (data):
#train_test_split

    
    stepwise_model_7 = auto_arima(data, start_p=1, start_q=1,
                           max_p=3, max_q=3, m=7,
                           start_P=0, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)
    
    return stepwise_model_7

In [124]:
orig_model_7 = sarima_model(ts.Close)


Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,1,1)[7]             : AIC=inf, Time=5.48 sec
 ARIMA(0,1,0)(0,1,0)[7]             : AIC=4795.849, Time=0.07 sec
 ARIMA(1,1,0)(1,1,0)[7]             : AIC=4275.548, Time=0.23 sec
 ARIMA(0,1,1)(0,1,1)[7]             : AIC=inf, Time=5.66 sec
 ARIMA(1,1,0)(0,1,0)[7]             : AIC=4793.468, Time=0.10 sec
 ARIMA(1,1,0)(2,1,0)[7]             : AIC=4009.461, Time=0.58 sec
 ARIMA(1,1,0)(2,1,1)[7]             : AIC=inf, Time=5.30 sec
 ARIMA(1,1,0)(1,1,1)[7]             : AIC=inf, Time=2.69 sec
 ARIMA(0,1,0)(2,1,0)[7]             : AIC=4013.685, Time=0.33 sec
 ARIMA(2,1,0)(2,1,0)[7]             : AIC=4008.358, Time=0.64 sec
 ARIMA(2,1,0)(1,1,0)[7]             : AIC=4274.611, Time=0.44 sec
 ARIMA(2,1,0)(2,1,1)[7]             : AIC=inf, Time=6.00 sec
 ARIMA(2,1,0)(1,1,1)[7]             : AIC=inf, Time=6.18 sec
 ARIMA(3,1,0)(2,1,0)[7]             : AIC=4010.353, Time=0.80 sec
 ARIMA(2,1,1)(2,1,0)[7]             : AIC=4010.334, Time=0.81 s

In [125]:
orig_pred_7 = orig_model_7.predict_in_sample(start=len(normalised_Data)-7, ends=len(normalised_Data))
orig_pred_31 = orig_model_7.predict_in_sample(start=len(normalised_Data)-31, ends=len(normalised_Data))

In [126]:
#reverse
#restored_Data = (normalised_future_forecast*fitted_lambda +1)**(1/fitted_lambda)

In [127]:
#evaluation of model fit
#boxljung test
#model_df is p+q
#m is seasonal period
#change boxpierce to true if you want to also run that test
#lbVal is the Ljung-Box test statistic and pVal is its p value
def getBoxLjung(data, lags, model_df, m):
    #demean the data
    demeanedData = data.sub(data.mean())
    lbVal, pVal = statsmodels.stats.diagnostic.acorr_ljungbox(demeanedData, lags, boxpierce = False, model_df = model_df, period = m)
    
    return lbVal, pVal
    

In [128]:
#Pass a fitted model
#returns 
#JBVal = Jarque-Bera test statistic
#JBPVal = pvalue of the test statistic
def getJarqueBera(data):
    
    JBVal, JBPVal, skewness, kurtosis = statsmodels.stats.stattools.jarque_bera(data.resid)
    
    return JBVal, JBPVal, skewness, kurtosis

## Function to sort different models:


In [129]:
import csv
import statsmodels

#input is a dataframe full of models
#I assume that there is no column that represents the skewness of model
def add_skewness(df, dataset):
    skewness = []
    #loop to get summary table of every model, then get skewness
    
    #To iterate through the df and isolate the model
    len = df.index
    for i in len:

        summary_table = df.model[i].summary()

        table_csv = summary_table.as_csv();

        skewness.append(table_csv[table_csv.find('Skew')+20 : table_csv.find('Skew')+25])


        #reader object stores all data
#         reader = csv.reader(table_csv)
        #change to finds out what cell skewness is in
#         skewness.append(table_csv[11][3])
        i+=1
    #at this point, skewness array should be full
    df['skewness'] = skewness
    return df;


In [130]:
add_skewness(df,ts.Close)
add_skewness(df_johnson,ts.Close)

,model,Order,SeasonalOrder,Aic,Bic,Hqic,Mse,heteroskedasticity,skewness
0,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(0, 1, 0)","(0, 1, 0, 7)",-12560.630180,-12555.026780,-12558.572989,0.003311,0.230721,-0.09
1,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(0, 1, 0)","(0, 1, 1, 7)",-13616.341315,-13605.134517,-13612.226934,0.003265,0.232281,-0.04
2,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(0, 1, 0)","(1, 1, 0, 7)",-13112.829293,-13101.622495,-13108.714912,0.003284,0.242139,-0.05
3,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(0, 1, 0)","(1, 1, 1, 7)",-13911.675568,-13894.865370,-13905.503995,0.003257,0.228486,-0.10
4,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(0, 1, 1)","(0, 1, 0, 7)",-12584.479638,-12573.272839,-12580.365256,0.003310,0.238035,-0.06
5,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(0, 1, 1)","(0, 1, 1, 7)",-13937.725989,-13920.915791,-13931.554417,0.003256,0.233142,-0.20
6,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(0, 1, 1)","(1, 1, 0, 7)",-13134.226562,-13117.416364,-13128.054990,0.003283,0.247177,-0.07
7,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(0, 1, 1)","(1, 1, 1, 7)",-13935.733200,-13913.319602,-13927.504436,0.003256,0.233213,-0.20
8,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(1, 1, 0)","(0, 1, 0, 7)",-12587.106650,-12575.899851,-12582.992268,0.003309,0.239686,-0.06
9,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(1, 1, 0)","(0, 1, 1, 7)",-13938.918779,-13922.108581,-13932.747206,0.003256,0.234046,-0.20


## Graphing some models:

In [131]:
# graphing 
def graphing(data, df):
    
    fig = px.line(data, x=ts.Close.index, y=[0])
    fig.data[0].line.color = 'rgb(0, 0, 0)'
    fig.data[0].name = "Original Data"
    fig.add_scatter
    
    fig.show()
    
    for i in range(6):
        model = sm.tsa.statespace.SARIMAX(normalised_Data, order = df['Order'][i], seasonal_order=df['SeasonalOrder'][i]).fit(disp=True)
        model = model.predict()
        fig = px.line(model, x=ts.Close.index, y=[0])
        fig.data[0].line.color = 'rgb(0, 0, 0)'
        fig.add_scatter
    
        fig.show()

In [132]:
graphing(normalised_Data, df)

C:\Users\Oisin Tong\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



C:\Users\Oisin Tong\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



In [133]:
model_i = [5,7,9]
model_i = df.shape[0]
print(model_i)

15


In [145]:
fig_7 = px.line(test_7, x=test_7.index, y=test_7, title="7-Day Prediction")
fig_7.data = []
fig_31 = px.line(test_31, x=test_31.index, y=test_31, title="31-Day Prediction")
fig_31.data = []

best_rmse_7 = 10000
best_rmse_7_johnson = 1000000
best_rmse_31 = 10000
best_rmse_31_johnson = 10000
preds = [None] * 4
for x in range(model_i):
    
    model = df.model[x]
    model_johnson = df_johnson.model[x]
    
    pred_7 = model.get_prediction(start=len(normalised_Data)-6, end=len(normalised_Data))
    pred_7_johnson = model_johnson.get_prediction(start=len(normalised_Data)-6, end=len(normalised_Data))
    pred_31 = model.get_prediction(start=len(normalised_Data)-30, end=len(normalised_Data))
    pred_31_johnson = model_johnson.get_prediction(start=len(normalised_Data)-30, end=len(normalised_Data))

    pred_7.predicted_mean = (pred_7.predicted_mean*fitted_lambda +1)**(1/fitted_lambda)
    pred_7_johnson.predicted_mean = (((johnson_lambda*pred_7_johnson.predicted_mean)+1)**(1/johnson_lambda)) - 1
    pred_31.predicted_mean = (pred_31.predicted_mean*fitted_lambda +1)**(1/fitted_lambda)
    pred_31_johnson.predicted_mean = (((johnson_lambda*pred_31_johnson.predicted_mean)+1)**(1/johnson_lambda)) - 1

    
    print("Predicted Vals of", x, ": \n", pred_7.predicted_mean)
    #print(model.summary_frame(alpha=0.05))   //mean isn't updated to reversed vals
    #conf_int = (model.conf_int(0.5)*fitted_lambda +1)**(1/fitted_lambda)   //don't think we should be reveresing the conf_int
    conf_int = (pred_7.conf_int(0.5))
    se = (pred_7.se_mean)
    rmse_7 = sm.tools.eval_measures.mse(test_7, pred_7.predicted_mean)
    rmse_7_johnson = sm.tools.eval_measures.mse(test_7, pred_7_johnson.predicted_mean)
    print(rmse_7_johnson)
    rmse_31 = sm.tools.eval_measures.mse(test_31, pred_31.predicted_mean)
    rmse_31_johnson = sm.tools.eval_measures.mse(test_31, pred_31.predicted_mean)

    print('\n',"Conf int: \n",conf_int,'\n\n')
    print('\n',"Std Err: \n",se,'\n\n')
    print('\n',"RMSE: \n",rmse_7,'\n\n')

    if(rmse_7 < best_rmse_7):
        preds[0] = pred_7.predicted_mean
    if(rmse_7_johnson < best_rmse_7_johnson):
        preds[1] = pred_7_johnson.predicted_mean
        
    if(rmse_31 < best_rmse_31):
        preds[2] = pred_31.predicted_mean
    if(rmse_31_johnson < best_rmse_31_johnson):
        preds[3] = pred_31_johnson.predicted_mean
#     if(rmse_31 < best_rmse_31):
#         fig_31.data = [fig_31.data[0]] #clear other traces
#         fig_31.add_scatter(x=test_31.index, y=pred_31.predicted_mean,
#             name='('+', '.join(str(e) for e in df.Order[x])+')'+' ('+', '.join(str(e) for e in df.SeasonalOrder[x])+')', mode='lines')
fig_7.add_scatter(x=test_7.index, y=test_7, name="Actual Values", mode='lines')
fig_7.add_scatter(x=test_7.index, y=orig_pred_7, name="Non-Transformed Prediction", mode='lines')
fig_7.add_scatter(x=test_7.index, y=preds[0], name="Box-Cox Prediction", mode='lines')
fig_7.add_scatter(x=test_7.index, y=preds[1], name="Yeo-Johnson Prediction", mode='lines')

fig_31.add_scatter(x=test_31.index, y=test_31, name="Actual Values", mode='lines')
fig_31.add_scatter(x=test_31.index, y=orig_pred_31, name="Non-Transformed Prediction", mode='lines')
fig_31.add_scatter(x=test_31.index, y=preds[2], name="Box-Cox Prediction", mode='lines')
fig_31.add_scatter(x=test_31.index, y=preds[3], name="Yeo-Johnson Prediction", mode='lines')

Predicted Vals of 0 : 
 [75.09690543 74.40079873 75.37361959 76.16918126 75.8096068  75.81395281
 75.75816225]
0.5253410689763022

 Conf int: 
 [[2.97047772 3.01713847]
 [2.96622128 3.01288202]
 [2.97215681 3.01881754]
 [2.97694393 3.02360466]
 [2.97478762 3.02144835]
 [2.97481375 3.02147449]
 [2.97447813 3.02113886]] 



 Std Err: 
 [0.01190347 0.01190347 0.01190347 0.01190347 0.01190347 0.01190347
 0.01190347] 



 RMSE: 
 0.5254719822580585 


Predicted Vals of 1 : 
 [75.06348623 75.53622359 76.08453097 76.14473846 76.39761218 76.80827227
 76.86836289]
0.02438163368329162

 Conf int: 
 [[2.97708068 3.01012895]
 [2.97994631 3.01299459]
 [2.98324361 3.01629188]
 [2.98360404 3.01665215]
 [2.98511387 3.01816197]
 [2.98755324 3.02060135]
 [2.9879089  3.020957  ]] 



 Std Err: 
 [0.00843084 0.00843084 0.00843084 0.00843079 0.00843079 0.00843079
 0.00843079] 



 RMSE: 
 0.006128043251695683 


Predicted Vals of 2 : 
 [75.11998649 74.73792006 75.52777365 76.0598215  75.81755302 76.7061885

In [146]:
fig_7.show()
fig_31.show()

## Dash App

In [136]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc

from pandas_datareader import DataReader # pip install pandas-datareader
from pandas_datareader import data

import pandas as pd
import pmdarima as pm
import plotly.graph_objects as go
import plotly.express as px


In [137]:
#set up app

app = dash.Dash()

In [138]:
#import datasets, for example;

start = pd.to_datetime('2010-01-01') # in YYYY-MM-DD format
end = pd.to_datetime('2018-01-01')
st = data.DataReader('NDAQ', 'yahoo', start , end) 


In [139]:
# dropdown options (currently runs through NASDAQ paramteres, should be changed to relevant parameters)

features = st.columns[1:-1]
opts = [{'label' : i, 'value' : i} for i in features]

# range slider options

st['High'] = pd.to_datetime(st.High)
dates = ['2015-02-17', '2015-05-17', '2015-08-17', '2015-11-17',
         '2016-02-17', '2016-05-17', '2016-08-17', '2016-11-17', '2017-02-17']


In [140]:
# create desired plotly figure (see https://images.plot.ly/plotly-documentation/images/python_cheat_sheet.pdf)

trace_1 = go.Scatter(x = st.Close, y = st['Close'],
                    name = 'Close',
                    line = dict(width = 2,
                                color = 'rgb(229, 151, 50)'))
layout = go.Layout(title = 'Time Series Plot',
                   hovermode = 'closest')
fig = go.Figure(data = [trace_1], layout = layout)

In [141]:
# create a Dash layout
app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = dbc.Container(dbc.Jumbotron(
    [
        html.H1("Time Series Analysis on Stocks Prediction", className="display-3"),
        html.P(
            "Group 40 ",
            className="lead",
        ),
        html.Hr(className="my-2"),
        html.P(
            "Lorem ipsum."
        ),
        dcc.Graph(id = 'actual', figure = actual),
        dcc.Graph(id = 'pred_7', figure = fig_7),
    ]
))


In [142]:
#run server

app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
